In [66]:
import pandas as pd
import glob
import os
import shutil


In [74]:
root = ".\Prepared"
for functionTypeDirectory in os.listdir(root):
    if os.path.isdir(os.path.join(root, functionTypeDirectory)):
        for functionDirectory in os.listdir(os.path.join(root, functionTypeDirectory)):
            for numberOfNodesDirectory in os.listdir(os.path.join(root, functionTypeDirectory, functionDirectory)):
                destinationPath =  os.path.join(root, functionTypeDirectory, functionDirectory, numberOfNodesDirectory, 'source-data')
                for nodeDirectory in os.listdir(destinationPath):
                    path =  os.path.join(destinationPath, nodeDirectory)
                    all_files = glob.glob(path + "/*.csv")
                    dfs = []
                    i = 1
                    for filename in all_files:
                        df = pd.read_csv(filename)
                        dfs.append(df)
                        if df['RAM'].mean() >= 2 and df['CPU'].head(15).mean() > 2:
                            if not os.path.exists(os.path.join(destinationPath.replace("Prepared", "Organised"), str(i))):
                                os.makedirs(os.path.join(destinationPath.replace("Prepared", "Organised"), str(i)))
                            shutil.copyfile(filename, os.path.join(destinationPath.replace("Prepared", "Organised"), str(i), os.path.basename(filename)))
                        i += 1
